In [ ]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 44.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv("/content/최종라벨링.csv", encoding='utf-8')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28740 entries, 0 to 28739
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  28740 non-null  object
 1   label    28740 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 449.2+ KB


In [ ]:
from konlpy.tag import Okt

okt = Okt()

In [ ]:
def tokenize(text):
    tokens = okt.morphs(text)
    return ' '.join(tokens)

In [ ]:
train_df, valid_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
train_df

,content,label
21547,옛날부터 그랬고 지금도 그러는구나...뭐 초등학교 급식 나눠주는곳에 아줌마들도 그러...,0
15757,우리도 눈이 있는데 저게 무슨 여자야;; 체형 옷색상 백퍼 남자임,1
22437,회수 못하냐? 돈있는늠이면 돈 도로 받으면 안돼?..,0
18283,중량스쿼드로 다져진 다리에 숏스웻팬츠가 진리,0
585,진짜 tmi다..,0
...,...,...
21575,이젠 아무대나 막 갖다붙이노.별게 다 성차별 ㅋㅋㅋㅋㅋㅋㅋㅋㅋ,0
5390,자기 남친 자랑하는거네글쓴여자 속마음은 난 내 남자친구가 청소하고 밥하고 설거지하고...,1
860,대깨삭들아ㅋ... 여기서부터 얼타고 있으면 다음주 다다음주..어떻게 보려고ㅋ... ...,1
15795,현실적인 엔딩이네 역겨운 맘충년들 삼대가 망해야 함,1


In [ ]:
train_df['tokenized_text'] = train_df['content'].apply(tokenize)
valid_df['tokenized_text'] = valid_df['content'].apply(tokenize)

#### 나이브베이즈 정리

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import resample
import matplotlib as mpl
from konlpy.tag import Okt
from transformers import BertTokenizer

In [ ]:
train_df

,content,label,tokenized_text
21547,옛날부터 그랬고 지금도 그러는구나...뭐 초등학교 급식 나눠주는곳에 아줌마들도 그러...,0,옛날 부터 그랬고 지금 도 그러는구나 ... 뭐 초등학교 급식 나눠주는 곳 에 아줌...
15757,우리도 눈이 있는데 저게 무슨 여자야;; 체형 옷색상 백퍼 남자임,1,우리 도 눈 이 있는데 저 게 무슨 여자 야 ;; 체형 옷 색상 백퍼 남자 임
22437,회수 못하냐? 돈있는늠이면 돈 도로 받으면 안돼?..,0,회수 못 하냐 ? 돈 있는 늠 이 면 돈 도로 받으면 안 돼 ?..
18283,중량스쿼드로 다져진 다리에 숏스웻팬츠가 진리,0,중량 스 쿼드로 다져진 다리 에 숏스웻팬츠 가 진리
585,진짜 tmi다..,0,진짜 tmi 다 ..
...,...,...,...
21575,이젠 아무대나 막 갖다붙이노.별게 다 성차별 ㅋㅋㅋㅋㅋㅋㅋㅋㅋ,0,이 젠 아무 대 나 막 갖다 붙이노 . 별 게 다 성차별 ㅋㅋㅋㅋㅋㅋㅋㅋㅋ
5390,자기 남친 자랑하는거네글쓴여자 속마음은 난 내 남자친구가 청소하고 밥하고 설거지하고...,1,자기 남친 자랑 하는거 네 글 쓴 여자 속마음 은 난 내 남자친구 가 청소 하고 밥...
860,대깨삭들아ㅋ... 여기서부터 얼타고 있으면 다음주 다다음주..어떻게 보려고ㅋ... ...,1,대 깨삭 들 아 ㅋ ... 여기 서 부터 얼 타고 있으면 다음주 다다 음주 .. 어...
15795,현실적인 엔딩이네 역겨운 맘충년들 삼대가 망해야 함,1,현실 적 인 엔딩 이네 역겨운 맘충년 들 삼대 가 망해야 함


In [ ]:
stopwords = set(['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '를', '으로', '자', '에', '와', '한', '하다'])

def remove_stopwords(text):
    tokens = text.split()
    tokens = [word for word in tokens if word not in stopwords]
    return ' '.join(tokens)

train_df['clean_text'] = train_df['tokenized_text'].apply(remove_stopwords)
valid_df['clean_text'] = valid_df['tokenized_text'].apply(remove_stopwords)

vectorizer = TfidfVectorizer()
clf = MultinomialNB()

model = make_pipeline(vectorizer, clf)

# train_df['label'] = train_df.iloc[:, 1:11].values.argmax(axis=1)
# valid_df['label'] = valid_df.iloc[:, 1:11].values.argmax(axis=1)
model.fit(train_df['clean_text'], train_df['label'])
predictions = model.predict(valid_df['clean_text'])
print("Classification Report:")
print(classification_report(valid_df['label'], predictions))

print("Accuracy Score:")
print(accuracy_score(valid_df['label'], predictions))


Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.23      0.37      1874
           1       0.72      0.99      0.84      3845
           2       0.00      0.00      0.00        29

    accuracy                           0.74      5748
   macro avg       0.55      0.41      0.40      5748
weighted avg       0.79      0.74      0.68      5748

Accuracy Score:
0.7383437717466945


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


---

In [ ]:
df1 = pd.read_csv('/content/최종라벨링.csv')
df2 = pd.read_csv('/content/malicious_comment_dataset_labeled.csv')

In [ ]:
df1['label'].value_counts()

label
1    19066
0     9525
2      149
Name: count, dtype: int64

In [ ]:
df2['label'].value_counts()

label
1    19165
0     9428
2      147
Name: count, dtype: int64

In [ ]:
df2

,content,label
0,이종석 한효주 나오는 드라마 이후로 드라마 안봤다. 2년전인가?? 좀 신선했었지. ...,1
1,씨바알..노무노무 술프노... 오늘 저녁은 꽂등심이다ㅠㅜ,1
2,짱깨 꺼라ㅡ패쓰,1
3,그들의 사생활 ~ 고인이된 설리를 위해서라도 모두 조용하길 지금 누굴 탓한다고 무슨...,0
4,아무리 법이 뭣같아도 무슨 자격으로 개인의 신상정보를 불특정 다수에게 공개하는지 도...,0
...,...,...
28735,저게 시대적언어면 한남충도 시대적언어 아니노 ㅋㅋ,1
28736,다른것보다 눈이 어떻게 저렇게 생기노.탄식.,1
28737,막노동을 해도 한국에살고말지 미쳤다고 남미를가냐?차라리 자살을하겠다.,1
28738,‘사형을 구형하였으나 여성인 점을 감안해 25년 선고’ ???내가 뭐 잘못본건가?개...,1


In [ ]:
df2 = df2[df2['label'] != 2]

In [ ]:
train_df, valid_df = train_test_split(df2, test_size=0.2, random_state=42)

train_df['tokenized_text'] = train_df['content'].apply(tokenize)
valid_df['tokenized_text'] = valid_df['content'].apply(tokenize)

stopwords = set(['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '를', '으로', '자', '에', '와', '한', '하다'])

def remove_stopwords(text):
    tokens = text.split()
    tokens = [word for word in tokens if word not in stopwords]
    return ' '.join(tokens)

train_df['clean_text'] = train_df['tokenized_text'].apply(remove_stopwords)
valid_df['clean_text'] = valid_df['tokenized_text'].apply(remove_stopwords)

vectorizer = TfidfVectorizer()
clf = MultinomialNB()

model = make_pipeline(vectorizer, clf)

model.fit(train_df['clean_text'], train_df['label'])
predictions = model.predict(valid_df['clean_text'])
print("Classification Report:")
print(classification_report(valid_df['label'], predictions))

print("Accuracy Score:")
print(accuracy_score(valid_df['label'], predictions))


Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.23      0.37      1841
           1       0.73      0.99      0.84      3878

    accuracy                           0.75      5719
   macro avg       0.83      0.61      0.61      5719
weighted avg       0.79      0.75      0.69      5719

Accuracy Score:
0.746983738415807


In [ ]:
df1['label'].value_counts()

label
1    19066
0     9525
2      149
Name: count, dtype: int64

In [ ]:
df2['label'].value_counts()

label
1    19165
0     9428
Name: count, dtype: int64

---
비악플 데이터 오버샘플링

In [ ]:
from imblearn.over_sampling import SMOTE

train_df, valid_df = train_test_split(df2, test_size=0.2, random_state=42)

train_df['tokenized_text'] = train_df['content'].apply(tokenize)
valid_df['tokenized_text'] = valid_df['content'].apply(tokenize)

stopwords = set(['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '를', '으로', '자', '에', '와', '한', '하다'])

def remove_stopwords(text):
    tokens = text.split()
    tokens = [word for word in tokens if word not in stopwords or word.isupper()]
    return ' '.join(tokens)

train_df['clean_text'] = train_df['tokenized_text'].apply(remove_stopwords)
valid_df['clean_text'] = valid_df['tokenized_text'].apply(remove_stopwords)

vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(train_df['clean_text'])
X_valid_vec = vectorizer.transform(valid_df['clean_text'])

smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_vec, train_df['label'])

print("SMOTE 적용 후 학습 데이터 (벡터):")
print(X_train_smote.toarray())
print("SMOTE 적용 후 학습 데이터 레이블:")
print(y_train_smote)

SMOTE 적용 후 학습 데이터 (벡터):
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
SMOTE 적용 후 학습 데이터 레이블:
0        1
1        1
2        1
3        1
4        0
        ..
30569    0
30570    0
30571    0
30572    0
30573    0
Name: label, Length: 30574, dtype: int64


In [ ]:
y_train_smote.value_counts()

label
1    15287
0    15287
Name: count, dtype: int64

In [ ]:
clf = MultinomialNB()
clf.fit(X_train_smote, y_train_smote)

predictions = clf.predict(X_valid_vec)

print("Classification Report:")
print(classification_report(valid_df['label'], predictions))

print("Accuracy Score:")
print(accuracy_score(valid_df['label'], predictions))

Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.62      0.68      1841
           1       0.83      0.91      0.87      3878

    accuracy                           0.81      5719
   macro avg       0.79      0.76      0.77      5719
weighted avg       0.81      0.81      0.81      5719

Accuracy Score:
0.8120300751879699


In [ ]:
def predict_comment(comment):
    tokenized_comment = tokenize(comment)
    clean_comment = remove_stopwords(tokenized_comment)
    comment_vec = vectorizer.transform([clean_comment])
    prediction = clf.predict(comment_vec)
    return "악성 댓글" if prediction == 1 else "클린 댓글"

test_comments = ["이거 정말 나빠요.",
                 "꺼져! 이런거 여기 쓰지마",
                 "왜 나쁜ㅅㄲ야.",
                 "오늘은 날씨가 좋아서 가시나들으면서 학교가야지",
                 "졸라 웃겨",
                 "ㅈㄴ 웃기는 소리하지마",
                 "웃기는 소리하고 앉았네",
                 "정현씨 못생겨서 보기 싫어요",
                 "정현이는 못생겨서 꼴보기 싫어"]

for text in test_comments:
    print(f"{text} : {predict_comment(text)}")

이거 정말 나빠요. : 악성 댓글
꺼져! 이런거 여기 쓰지마 : 악성 댓글
왜 나쁜ㅅㄲ야. : 악성 댓글
오늘은 날씨가 좋아서 가시나들으면서 학교가야지 : 악성 댓글
졸라 웃겨 : 악성 댓글
ㅈㄴ 웃기는 소리하지마 : 악성 댓글
웃기는 소리하고 앉았네 : 악성 댓글
정현씨 못생겨서 보기 싫어요 : 악성 댓글
정현이는 못생겨서 꼴보기 싫어 : 악성 댓글
